In [1]:
using DataFrames, LinearAlgebra, Distributions, Revise, Serialization, DebuggingUtilities

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

---

## Computation

In [31]:
Threads.nthreads()

32

In [9]:
test1state = false
if test1state 
    dirpath = "/export/storage_adgandhi/MiscLi/factract/analysis"
else
    dirpath = "jls"
end;
ec = deserialize("$dirpath/ec.jls");

In [10]:
ec = deserialize("$dirpath/ec.jls");
pars = deserialize("$dirpath/pars.jls");
deltas, q_iter = m.compute_deltas(ec, pars, σ,max_iter=10000, verbose=true);
# serialize("jls/deltas.jls", deltas);

dist = NaN, iterations = 454


In [13]:
using BenchmarkTools

In [73]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  0.274327 seconds (519.15 k allocations: 61.624 MiB, 29.17% compilation time)


In [74]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 10
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  0.076545 seconds (900.14 k allocations: 101.865 MiB)


In [75]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1000
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  1.515471 seconds (22.36 M allocations: 1.873 GiB, 5.73% gc time)


In [17]:
Threads.nthreads()

16

In [18]:
# # this was the line that the memory profiler said was creating the vast majority of allocations
# # ...which doesn't make sense because it's not even in the inner loop
# # ...and it's the inner loop that's driving the allocations and times (according to benchmarks)
# @time for t in ec.tracts
#     t.abδ .= t.D * pars.nlcoefs
# end

___
continued in `gmm.ipynb`